In the reactions forming our final product of Eicosapentaenoic Acid, NADPH is used several times as H donor. To increase yield, a higher available concentration of NADPH might allow for the reactions to run faster and result in more EDA. To test this theory, cofactor swapping was used to find reactions where NADPH could be exchanged for NADH, as to increase the amount of NADPH available for product reactions. 

In [1]:
#requirements
pip install cobra==0.20.0
pip install cameo
pip install markupsafe==2.0.1


Note: you may need to restart the kernel to use updated packages.


In [2]:
#load model
%run Scaffold_Y_lipo.ipynb

In [3]:
#medium is defined here?
medium = model.medium

medium['EX_so4_e'] = 1000
medium['EX_pi_e'] = 1000
medium['EX_o2_e'] = 1000
medium['EX_nh4_e'] = 1000
medium['EX_glc__D_e'] = 300
medium['EX_h2o_e'] = 1000
medium['EX_glyc_e'] = 1000

model.medium = medium
model.medium



{'EX_nh4_e': 1000,
 'EX_o2_e': 1000,
 'EX_pi_e': 1000,
 'EX_so4_e': 1000,
 'EX_glyc_e': 1000,
 'EX_glc__D_e': 300,
 'EX_h2o_e': 1000}

In [6]:
from collections.abc import Mapping

In [7]:
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization, NADH_NADPH

#parameters for the function, defining biomass reaction, the reaction to be optimized

model.solver = "glpk" 
biomass = model.reactions.BIOMASS_SC4_bal
biomass.lower_bound = 0.1 
demand = model.reactions.DM_Eicosapentaenoic_Acid

model.objective = demand
product_yield = product_yield(demand, model.reactions.EX_glc__D_e)

In [15]:
#running CofactorSwapOptimization
swap_opt = CofactorSwapOptimization(model=model, objective_function=product_yield, plot=True, cofactor_id_swaps=(['nadp_c', 'nadph_c'], ['nad_c', 'nadh_c']))

In [16]:

result = swap_opt.run(max_size=5)

Starting optimization at Sun, 03 Dec 2023 14:58:15


HBox()

Finished after 00:00:31


In [17]:
#Results
result

,index,targets,fitness
0,0,"(D9E, GAPD)",0.961075
1,1,"(GAPD,)",0.947726
2,2,"(MDH, D8ds)",0.917238
3,3,"(FAS160, MDH)",0.913101
4,4,"(FAS80COA_L, D5ds)",0.905296
5,5,"(MDH, IMPD)",0.904834
6,6,"(MDH,)",0.904833


The reactions found to be using NADPH as a cofactor are shown here. After finding the reactions in the model, some of our inserted reactions that use NADPH (D9E, D8ds and D5ds) were found, however not all of them which is unexpected, as all the added reactions use either NADPH or NADP. The remaining reactions are IMP dehydrogenase, Malate dehydrogenase and Glyceraldehyde dehydrogenase, as well as general fatty acid synthesis reactions that precede EPA formation. To increase the flux through these reactions, analysis could be done to find limiting steps, and change the cofactor to NADH instead, as it is possible that the overreliance on NADPH thorughout these reactions are rate limiting. 

In [24]:
model.reactions.IMPD

Reaction identifier,IMPD
Name,IMP dehydrogenase
Memory address,0x019dd5384048
Stoichiometry,h2o_c + imp_c + nad_c --> h_c + nadh_c + xmp_c H2O H2O + IMP C10H11N4O8P + Nicotinamide adenine dinucleotide --> H+ + Nicotinamide adenine dinucleotide - reduced + Xanthosine 5'-phosphate
GPR,YAR073W or YML056C or YLR432W or YHR216W or YAR075W
Lower bound,0.0
Upper bound,999999.0


In [20]:
model.reactions.GAPD

Reaction identifier,GAPD
Name,Glyceraldehyde-3-phosphate dehydrogenase
Memory address,0x019dd5223288
Stoichiometry,g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c Glyceraldehyde 3-phosphate + Nicotinamide adenine dinucleotide + Phosphate <=> 3-Phospho-D-glyceroyl phosphate + H+ + Nicotinamide adenine dinucleotide - reduced
GPR,YJL052W or YGR192C or YJR009C
Lower bound,-999999.0
Upper bound,999999.0


In [26]:
model.reactions.MDH

Reaction identifier,MDH
Name,Malate dehydrogenase
Memory address,0x019dd4ce9f88
Stoichiometry,mal__L_c + nad_c <=> h_c + nadh_c + oaa_c L-Malate + Nicotinamide adenine dinucleotide <=> H+ + Nicotinamide adenine dinucleotide - reduced + Oxaloacetate
GPR,YOL126C
Lower bound,-999999.0
Upper bound,999999.0


In [27]:
model.reactions.FAS160

Reaction identifier,FAS160
Name,Fatty acid synthase n C160
Memory address,0x019dd5157b88
Stoichiometry,3.0 h_c + malcoa_c + 2.0 nadph_c + ttdca_c --> co2_c + coa_c + h2o_c + hdca_c + 2.0 nadp_c 3.0 H+ + Malonyl CoA C24H33N7O19P3S + 2.0 Nicotinamide adenine dinucleotide phosphate - reduced + Tetradecanoate (n-C14:0) --> CO2 CO2 + Coenzyme A + H2O H2O + Hexadecanoate (n-C16:0) + 2.0...
GPR,YGR037C and YKL182W and YPL231W and YNR016C
Lower bound,0.0
Upper bound,999999.0


In [28]:
model.reactions.FAS80COA_L

Reaction identifier,FAS80COA_L
Name,Fatty acyl CoA synthase n C80CoA lumped reaction
Memory address,0x019dd518f788
Stoichiometry,accoa_c + 9.0 h_c + 3.0 malcoa_c + 6.0 nadph_c --> 3.0 co2_c + 3.0 coa_c + 3.0 h2o_c + 6.0 nadp_c + occoa_c Acetyl-CoA + 9.0 H+ + 3.0 Malonyl CoA C24H33N7O19P3S + 6.0 Nicotinamide adenine dinucleotide phosphate - reduced --> 3.0 CO2 CO2 + 3.0 Coenzyme A + 3.0 H2O H2O + 6.0 Nicotinamide adenine...
GPR,YGR037C and YKL182W and YPL231W and YNR016C
Lower bound,0.0
Upper bound,999999.0
